# Selecting identifiable parameters

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# from functools import partial
from model import NeoHookeanSolutionGenerator

%matplotlib inline

### We first fix the parameters without any variability

In [3]:
# As determined from experiments
omega_val = np.pi
L_val = 2.0
zeta_val = 0.5
nu_f_val = 0.125

generator = NeoHookeanSolutionGenerator(omega_val, L_val, zeta_val, nu_f_val)

###  These parameters however can vary

In [4]:
n_params = 3
c1_val = 0.0625
V_wall_val = 1.0
nu_s_val = 0.01

v_f, v_s = generator.generate_velocities_for(c1_val, V_wall_val, nu_s_val)

In [5]:
res_y = 100
res_t = 20
Y = np.linspace(0, L_val / 2.0, res_y)
# offset = np.pi / 2
offset = 0.0
T = np.linspace(
    0 + offset / omega_val, (2 * np.pi + omega_val) / omega_val, res_t, endpoint=False
)
y_stations = np.array([0.2, 0.4, 0.6, 0.8])

### Uncomment in case you want to see the velocity response

In [6]:
# for ts in T:
#     # print()
#     solid_velocity = (Y < 0.5) * (v_s(Y, ts * np.ones_like(Y)))
#     fluid_velocity = (Y >= 0.5) * (v_f(Y, ts * np.ones_like(Y)))
#     vel_comb = solid_velocity + fluid_velocity
#     # print(ts * omega / (2 * np.pi), np.max(np.abs(solid_velocity)), np.max(np.abs(fluid_velocity)))
#     plt.plot(vel_comb, Y, linewidth=3)
# # plt.plot(np.linspace(-V_wall_val, V_wall_val, 10), L_s * np.ones(10), '-.', c="k")

In [7]:
# offset = 0.0
# res_t = 20
# T = np.linspace(0 + offset / omega_val, (2 * np.pi) / omega_val, res_t, endpoint=False)
# y_stations = np.array([0.2, 0.4, 0.6, 0.8])
# y_profiles = np.zeros((len(y_stations), len(T)))

# for i_station, y_station in enumerate(y_stations):
#     solid_velocity = v_s(y_station, T)
#     fluid_velocity = v_f(y_station, T)
#     solid_mask = (y_station < 0.5)
#     y_profiles[i_station] = solid_mask * solid_velocity + (1.0 - solid_mask) * fluid_velocity

In [8]:
# for profile in (y_profiles):
#     plt.plot(T, profile, linewidth=3)

## Parameter identifiability for one set of parameters, across different y and T

In [9]:
chi_f, chi_s = generator.generate_sensitivities_for(c1_val, V_wall_val, nu_s_val)

In [10]:
chi_matrix = np.zeros((len(y_stations) * len(T), n_params))

for i_station, y_station in enumerate(y_stations):
    solid_sens = chi_s(y_station, T).squeeze().T  # (len(T), 3)
    fluid_sens = chi_f(y_station, T).squeeze().T  # (len(T), 3)
    solid_mask = y_station < 0.5
    chi_matrix[i_station * len(T) : (i_station + 1) * len(T)] = (
        solid_mask * solid_sens + (1.0 - solid_mask) * fluid_sens
    )

In [11]:
chi_matrix.shape

(80, 3)

In [16]:
from scipy.linalg import qr
Q, R, P = qr(chi_matrix, pivoting=True)
# Q

In [17]:
np.linalg.matrix_rank(Q)

80

## Parameter identifiability across a range of parameters

In [18]:
# c1_val = 0.0625
# V_wall_val = 1.0
# nu_s_val = 0.0
from itertools import product

c1_vals = [0.03125, 0.0625, 0.125]
V_wall_vals = [0.8, 0.9, 1.0]
nu_s_vals = [0.0, 0.00625, 0.0125]

chi_matrix_across_params = np.zeros((3 * 3 * 3, n_params))

y_station = 0.2
t_station = 1.0

for i_p, (p_c1_val, p_V_wall_val, p_nu_s_val) in enumerate(product(c1_vals, V_wall_vals, nu_s_vals)): 
    chi_f, chi_s = generator.generate_sensitivities_for(p_c1_val, p_V_wall_val, p_nu_s_val)
    
    # for i_station, y_station in enumerate(y_stations):
    solid_sens = chi_s(y_station, t_station).squeeze().T  # (len(T), 3)
    fluid_sens = chi_f(y_station, t_station).squeeze().T  # (len(T), 3)
    solid_mask = y_station < 0.5
    chi_matrix_across_params[i_p] = (
        solid_mask * solid_sens + (1.0 - solid_mask) * fluid_sens
    )

In [19]:
Q, R, P = qr(chi_matrix_across_params.T, pivoting=True)
Q

array([[-9.01635638e-01, -4.32495437e-01,  9.35010885e-04],
       [ 3.67871432e-03, -5.50727095e-03,  9.99978068e-01],
       [-4.32480802e-01,  9.01619303e-01,  6.55657891e-03]])

As we see above, Q is full rank. We reconfirm this using a rank-revealing SVD, via numpy `matrix_rank`.

In [20]:
np.linalg.matrix_rank(chi_matrix_across_params.T)

3